<img src="../images/starwarsstartrek.png.crdownload" style="float: center; margin: 0px; height: 250px; width: 100%">



# Project 3: Web APIs & NLP

#### _Author: Afolabi Cardoso_

---

#### Overview
This is a natural language processing project. The aim of the project is to build and compare two classification models; logistic regression and random forest. 

The models are developed to accurately predict which subreddit a post belongs to.

For this project, I am using the [Star Trek](https://www.reddit.com/r/startrek/) and [Star Wars](https://www.reddit.com/r/StarWars/) subreddit.

I used the  [Pushshift's](https://github.com/pushshift/api) API to collect 1000 posts each from both subreddits.


---
#### Contents:
[Overview](#Overview) | [Imports](#Imports) | [Data Cleaning Training Set](#Data-Cleaning-Training-Set) | [Data Cleaning Test Set](#Data-Cleaning-Test-Set) | [Exports](#Exports)


#### Imports

In [18]:
import pandas as pd
import numpy as np
import requests
import time

#### Data Collecting`

I am going to create a **get_post** function. The function accepts two parameters. The first parameter is a list containing the names of the subreddits and the second parameter is the number of post I want to collect.

The functions returns a list of dataframes containing all the requested posts.

In [19]:
#function definition get_post

def get_post(lst, no_post):
    url = 'https://api.pushshift.io/reddit/search/submission' #pushshift reddit API link   
    posts = []
    
    for sub in lst:
        post_count = no_post   #assign the number of post passed to post_count
        before  = int(time.time())  #get the current time and assigns it to before
        
        while post_count > 0:
                
            if post_count < 100:   
                size = post_count
            else:
                size = 100
            
            params = {                      #parameters for the get method
                'subreddit': sub,
                 'size' : size,
                 'before': before
             }

            res = requests.get(url, params)
            json_data = res.json()
            df_data = pd.DataFrame(json_data['data'])
            
            posts.append(df_data)
            
            post_count -= 100 
            before = df_data['created_utc'].iloc[-1]
            
        time.sleep(3)  #create a 3 seconds wait between pulls
    return posts

I am going to call the **get_post** and pass in the subreddits and number of posts. I will collect 1010 post, the extra 10 post is for if I need to replace posts with errors.

In [27]:
subreddits = ['StarWars', 'startrek']
#df_list = get_post(subreddits, 1010)

Concat the list of dataframes

In [28]:
#df = pd.concat(df_list, ignore_index=True)

In [29]:
df[['title','selftext','created_utc','subreddit']]

,title,selftext,created_utc,subreddit
0,No wonder Ukraine has lasted so long.,,1646191472,StarWars
1,SO EXCITED FOR IT!!,,1646191396,StarWars
2,"“Oh, I’m afraid the deflector shield will be q...",,1646191081,StarWars
3,Darth Vade Vs. General Grevious,,1646190952,StarWars
4,Satine was a terrible ruler,I know this sounds a bit harsh but let me get ...,1646190846,StarWars
...,...,...,...,...
2014,Calypso question.,I am assuming that this episode takes place 10...,1644265266,startrek
2015,I know what is behin the DMA,Another sad Kelpian.,1644264724,startrek
2016,DS9 Klingon language captions,I recently started a rewatch of DS9 and “Blood...,1644260603,startrek
2017,Star Trek Portraits as NFTs by Jeffery Newbury,[removed],1644257811,startrek


In [32]:
df.groupby('subreddit').count() [['selftext','title','created_utc']]

,selftext,title,created_utc
subreddit,,,
StarWars,1008,1010,1010
startrek,1008,1009,1009


In [38]:
df.to_csv('../data/raw.csv', index=False)